# US Seaport Volumes

---
#### For the top 10 ports (80% of all TEUs)

Note: TEU means "Twenty-foot [container] Equivalent Unit"

A look at shipping port traffic, according to TEU reports from each of the top 10 port authorities.

In [1]:
import pandas as pd
import altair as alt
import numpy as np
import re
from json import loads, JSONDecodeError
from io import BytesIO

from time import sleep
from joblib import Memory

from urllib.request import urlopen, Request
from urllib.error import HTTPError, URLError

from tabula import read_pdf

from google.google import search as googleSearch

In [2]:
def doChart(df, location='Oakland', metric='Full-Total'):
    #metric = 'Full-Total'

    return alt.Chart(df.reset_index()[['dt', metric]].dropna()).mark_line(color='purple').encode(
        alt.X('dt:T', axis=alt.Axis(title='')),
        alt.Y(f'{metric}:Q', axis=alt.Axis(title='Volume [TEUs]')),
        tooltip=[alt.Tooltip("dt:T", format="%b %Y"), alt.Tooltip(f"{metric}:Q", format=",.02f")]
    ).properties(
        title=f'Port of {location} {metric} Container Volume',
        width=750,
        height=400
    )

def doYoyChart(df, location='Oakland', metric='Full-Total'):
    df_yoy = df.reset_index()[['dt', metric]].dropna().set_index('dt').sort_index().pct_change(12).apply(lambda v: v * 100).reset_index()

    return alt.Chart(df_yoy).mark_bar(width=2, color='blue').encode(
        alt.X('dt:T', axis=alt.Axis(title='')),
        alt.Y(f'{metric}:Q', axis=alt.Axis(title='Year-over-Year Volume Growth [%]')),
        color=alt.condition(f"datum['{metric}'] < 0",
            alt.value('lightsalmon'),
            alt.value('royalblue')
        ),
        tooltip=[alt.Tooltip("dt:T", format="%b %Y"), alt.Tooltip(f"{metric}:Q", format=",.02f")]
    ).properties(
        title=f'Port of {location} {metric} Container Volume Growth',
        width=750,
        height=400
    )

uaString = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'

memory = Memory('data/', verbose=0)

## Port of Los Angeles

In [3]:
# Port of Los Angeles
# https://www.portoflosangeles.org/business/statistics/container-statistics

# 1995
# https://www.portoflosangeles.org/business/statistics/container-statistics/historical-teu-statistics-2019

@memory.cache
def fetchLaData(u, year, cols, args = { 'skiprows': 1 }):
 
    req = Request(u, data=None, headers={ 'User-Agent': uaString })

    data = urlopen(req)

    tmp = pd.read_html(BytesIO(data.read()), **args)[0]
    try:
        tmp.columns = ['Month'] + cols
    except ValueError as e:
        #print(tmp.head())
        print(e)
        
    tmp['dt'] = tmp['Month'].apply(lambda v: pd.to_datetime("{}-{}".format(v, year), format="%B-%Y", errors='coerce'))
    
    return tmp

def fetchLaDataNoCache(u, year, cols, args = { 'skiprows': 1 }):
 
    req = Request(u, data=None, headers={ 'User-Agent': uaString })

    data = urlopen(req)

    tmp = pd.read_html(BytesIO(data.read()), **args)[0]
    try:
        tmp.columns = ['Month'] + cols
    except ValueError as e:
        #print(tmp.head())
        print(e)
        
    tmp['dt'] = tmp['Month'].apply(lambda v: pd.to_datetime("{}-{}".format(v, year), format="%B-%Y", errors='coerce'))
    
    return tmp

colsPre96 = ['Loaded Imports', 'Empty Imports', 'Total Imports', 
        'Loaded Export', 'Empty Exports', 'Total TEUs', 'Prior Year Change']

cols = ['Loaded Imports', 'Empty Imports', 'Total Imports', 
        'Loaded Export', 'Empty Exports', 'Total Exports', 'Total TEUs', 
        'Prior Year Change']

args = {
    'skiprows': 1
}

urlFor = lambda y: "https://www.portoflosangeles.org/business/statistics/container-statistics/historical-teu-statistics-{}".format(y)

dfs_la = []
for y in range(1995, 2021):
    url = urlFor(y)
    #print(url)
    if y < 1996:
        dfs_la.append(fetchLaData(url, y, colsPre96, args))
    elif y == 2020:
        dfs_la.append(fetchLaDataNoCache(url, y, cols, {}))
    else:
        dfs_la.append(fetchLaData(url, y, cols, {}))

df_la = pd.concat(dfs_la).dropna() #.tail()

df_la[cols[:1]] = df_la[cols[:1]].apply(pd.to_numeric)

for c in df_la.columns:
    if c in ['Month', 'dt', 'Prior Year Change']:
        continue
    df_la[c] = df_la[c].apply(float)

#df_la.head()

#df_la_smoothed = df_la.set_index('dt').resample('2M').mean().reset_index()

In [4]:
c = doYoyChart(df_la, location='Los Angeles', metric='Loaded Imports')
c.save('transportation-ocean.png')
c.display()

alt.Chart(...)

In [5]:
doYoyChart(df_la, location='Los Angeles', metric='Loaded Export')

alt.Chart(...)

In [6]:
doYoyChart(df_la, location='Los Angeles', metric='Total Imports')

alt.Chart(...)

In [7]:
doYoyChart(df_la, location='Los Angeles', metric='Total TEUs')

alt.Chart(...)

## Port of Long Beach

In [8]:
# http://www.polb.com/economics/stats/teus_archive.asp

uaString = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'

req = Request("https://thehelm.polb.com/stellar_custom_table/table100/", data=None, headers={ 'User-Agent': uaString })

data = urlopen(req)

df_lb_raw = pd.read_html(BytesIO(data.read()), displayed_only=False)[0].iloc[:-1]

df_lb_raw = df_lb_raw.iloc[:, 1:]

df_lb_raw[df_lb_raw.columns[1:]] = df_lb_raw[df_lb_raw.columns[1:]].apply(pd.to_numeric)

df_lb_raw['dt'] = df_lb_raw['Date'].map(lambda v: pd.to_datetime(v, format='%b %Y'))

#df_lb_raw.head(16)

In [9]:
doYoyChart(df_lb_raw, location='Long Beach', metric='Total Loaded')

alt.Chart(...)

In [10]:
doYoyChart(df_lb_raw, location='Long Beach', metric='Total')

alt.Chart(...)

In [11]:
doYoyChart(df_lb_raw, location='Long Beach', metric='Empties')

alt.Chart(...)

In [12]:
doYoyChart(df_lb_raw, location='Long Beach', metric='Loaded Inbound')

alt.Chart(...)

## Port of New York / New Jersey

In [13]:
dataUrl = "https://www.panynj.gov/content/port/en.model.json"

#jqPath = '\'.":children"."/port/en/our-port/facts-and-figures".":items"."root".":items"."responsivegrid".":items"."accordionlist_435572874".":items"\''
    
def doNynjFetch(retries = 5):
    if retries < 0:
        return loads("{}")
    try:
        req = Request(dataUrl, data=None, headers={ 'User-Agent': uaString })

        data = urlopen(req)

        dict_data = loads(data.read())

        r = dict_data[":children"]\
                     ["/port/en/our-port/facts-and-figures"]\
                     [":items"]\
                     ["root"]\
                     [":items"]\
                     ["responsivegrid"]\
                     [":items"]\
                     ["accordionlist_435572874"]\
                     [":items"]
        
        return r
    except JSONDecodeError:
        sleep(1)
        return doNynjFetch(retries-1)

nynj_dict_content = doNynjFetch()
    
post2011Cols = ['Import Loads', 'Import Empties', 'Export Loads', 'Export Empties', 'Total Loads', 'Total Empties', 'Total TEUs', 'Total Rail Lifts']
cols = ["Import TEUs", "Export TEUs", "Total TEUs", "Total Rail Lifts"]

dfs_nynj = []
for v in nynj_dict_content.values():
    year = v["linkAriaLabel"]
    #print(year)
    try:
        tmp = pd.read_html(v[":items"]["text"]["text"])
    except KeyError:
        # text_copy
        tmp = pd.read_html(v[":items"]["text_copy"]["text"])
    for n, monthData in enumerate(tmp):
        # skip the YTD section
        #if n < 2:
        #    continue
        
        mtmp = monthData.copy()
        try:
            mtmp.columns = ['Period'] + post2011Cols
            
            # cleanup types before doing math
            mtmp[post2011Cols] = mtmp[post2011Cols].applymap(lambda v: re.sub("[^0-9.]", "", str(v)))
            mtmp[post2011Cols] = mtmp[post2011Cols].apply(lambda v: pd.to_numeric(v, errors='coerce'))
            
            mtmp['Import TEUs'] = mtmp['Import Loads'] + mtmp['Import Empties']
            mtmp['Export TEUs'] = mtmp['Export Loads'] + mtmp['Export Empties']
            
            # Monthy is shown Dec->Jan after 2014, Jan->Dec before 2014 (and for 2020)
            index = 13-n if int(year) > 2014 and int(year) != 2020 else n
            mtmp['dt'] = mtmp.iloc[:, 0].map(lambda v: pd.to_datetime("{}-{}".format(index, v), format="%m-%Y", errors='coerce'))
        except ValueError:
            mtmp.columns = ['Period'] + cols
            mtmp['dt'] = mtmp.iloc[:, 0].map(lambda v: pd.to_datetime(v, format="%B %y", errors='coerce'))
        
            mtmp[cols] = mtmp[cols].applymap(lambda v: re.sub("[^0-9.]", "", str(v)))
            mtmp[cols] = mtmp[cols].apply(lambda v: pd.to_numeric(v, errors='coerce'))
        
        dfs_nynj.append(mtmp)

commonFields = ['Period', 'dt', 'Import TEUs', 'Export TEUs', 'Total TEUs']

df_nynj = pd.concat(dfs_nynj)[commonFields].dropna() #.tail(30)

df_nynj = df_nynj.groupby('dt').agg('mean') #.head()

In [14]:
doChart(df_nynj.reset_index(), location="New York/New Jersey", metric="Import TEUs")

alt.Chart(...)

In [15]:
doYoyChart(df_nynj.reset_index(), location="New York/New Jersey", metric="Import TEUs")

alt.Chart(...)

In [16]:
doYoyChart(df_nynj.reset_index(), location="New York/New Jersey", metric="Export TEUs")

alt.Chart(...)

In [17]:
doYoyChart(df_nynj[12:].reset_index(), location="New York/New Jersey", metric="Total TEUs")

alt.Chart(...)

## Georgia Ports

In [18]:
@memory.cache
def doGaFetch(url, retries=5):
    if retries < 0:
        return None
    try:
        req = Request(url, data=None, headers={ 'User-Agent': uaString })

        data = urlopen(req)

        tabs = read_pdf(BytesIO(data.read()), pages='all')

        # noop to test for the table
        tabs[0]
        
        return tabs
    except KeyError:
        sleep(1)
        return doGaFetch(url, retries-1)
    
reportPage = 'https://gaports.com/sales/by-the-numbers/'
currentReport = !curl -s -L $reportPage | egrep -i 'Monthly.*TEU.*pdf' | perl -pe 's/.*href=\"([^"]+)\".*/\1/'

#print(currentReport[0])

tabs = doGaFetch(currentReport[0])

In [72]:
tabs[0]

,Unnamed: 0,Unnamed: 1,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Total
0,NaN,2020,"377,671","364,405","335,789","337,890","337,359","338,287","360,697",NaN,NaN,NaN,NaN,NaN,"2,452,098"
1,NaN,2019,"430,079","312,042","410,326","364,481","373,394","361,906","387,022","437,747","369,999","428,381","362,964","360,834","4,599,172"
2,NaN,2018,"338,793","341,094","355,208","356,717","361,029","370,726","378,767","375,844","364,150","413,778","344,506","351,366","4,351,976"
3,NaN,2017,"331,468","330,539","311,770","333,006","350,104","337,711","336,099","348,297","325,141","409,814","309,147","323,117","4,046,212"
4,NaN,2016,"285,301","307,035","295,149","298,040","313,485","288,364","314,714","330,846","308,348","310,393","300,671","292,173","3,644,519"
5,NaN,2015,"293,654","284,037","333,056","335,907","338,207","330,420","324,242","315,175","317,417","321,094","284,146","260,074","3,737,427"
6,NaN,2014,"259,091","248,750","260,559","266,935","290,436","268,119","293,889","301,822","295,698","311,760","271,357","277,634","3,346,048"
7,NaN,2013,"230,372","242,425","232,545","258,951","261,520","265,787","246,652","280,813","261,821","274,327","243,215","235,300","3,033,727"
8,NaN,2012,"247,102","237,076","259,795","248,911","256,530","258,950","252,130","270,610","257,159","230,640","228,184","219,128","2,966,213"
9,NaN,2011,"236,981","235,643","238,013","260,063","254,330","245,544","262,723","237,792","269,856","255,116","234,150","214,472","2,944,681"


In [ ]:
cols = ['Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Total']

flatten = lambda l: [item for sublist in l for item in sublist]

def parseGpTable(df, columns, start=0, end=17):
    gp_raw = df.iloc[start:end, 1:].copy() if end is not None else df.iloc[start:17, :].copy()

    #print(gp_raw.iloc[0, :].values)
    
    firstRowVals = flatten(map(lambda v: str(v).split(), gp_raw.iloc[0, :-1].dropna().values))
    #print(firstRowVals)
    
    # Use all the month values in order then append the total value as the last element
    firstRow = [firstRowVals[n] if n < len(firstRowVals) else '' for n in range(len(cols[:-1]))] + [gp_raw.iloc[0, -1]]
    #print(firstRow)

    records = gp_raw.iloc[:, 1:].apply(lambda s: flatten(map(lambda v: str(v).split(), s.values)), axis=1)

    #print(records.head(1))
    
    df_ga_melted = pd.DataFrame.from_records([firstRow] + [r for r in records.values], columns=columns).melt(id_vars='Year')
    
    #print(df_ga_melted[df_ga_melted['Year'] == '2020'].head(15))
    
    df_ga_melted = df_ga_melted[df_ga_melted['variable'] != 'Total']
    df_ga_melted['dt'] = pd.to_datetime(df_ga_melted['Year'] + '-' + df_ga_melted['variable'], format='%Y-%b')
    df_ga_melted['value'] = df_ga_melted['value'].apply(lambda v: pd.to_numeric(str(v).replace(",", ""), errors='coerce'))

    df_ga_tst = df_ga_melted[['value', 'dt']].set_index('dt').resample('1M').mean().rename({'value': 'TEU Total'}, axis=1)

    return df_ga_tst.reset_index()

df_ga_tst = parseGpTable(tabs[0], columns=cols, start=3, end=None)

In [30]:
doYoyChart(df_ga_tst, location='Georgia', metric='TEU Total')

alt.Chart(...)

In [75]:
cols2 = ['Year', 'Key', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Total']

#gp_raw = tabs[0].iloc[3:21].copy()

gp_detail_raw = pd.concat([tabs[0].iloc[18:].copy()] + tabs[1:]).copy()

gp_detail_raw = gp_detail_raw[~gp_detail_raw.iloc[:, 1].isna()]

#print(gp_detail_raw)

# Exclude rows with erroneous data
gp_detail_raw = gp_detail_raw[~gp_detail_raw.iloc[:, 2].str.contains("%")]
gp_detail_raw = gp_detail_raw[gp_detail_raw.iloc[:, 1].str.match("^(I|E|T|\d)")]

#print(gp_detail_raw.head(25))

hypenate = lambda v: str(v).replace("rt ", "rt-")\
                           .replace("Total Full", "Total-Full")\
                           .replace("Total Empty", "Total-Empty")\
                           .replace("Total All", "Total-All")

gp_detail_raw.iloc[:, 1] = gp_detail_raw.iloc[:, 1].apply(hypenate)

"""
# Special case for the current year
firstRowVals2 = flatten(map(lambda v: hypenate(v).split(), gp_detail_raw.iloc[0, :-1].values))
firstRow2 = [firstRowVals2[n] if n < len(firstRowVals2) else '' for n in range(len(cols2[:-1]))] + [gp_detail_raw.iloc[0, -1]]

# General case for the historical years
remainingRows = gp_detail_raw.iloc[1:].apply(lambda s: flatten(map(lambda v: hypenate(v).split(), s.to_list())), axis=1)

gp_detail = pd.DataFrame.from_records([firstRow2] + [r for r in remainingRows.values], columns=cols2)

#print(gp_detail.head())

# Find all the records that don't have the year in them
rowsToShift = gp_detail[~gp_detail['Year'].str.match("^\d")].index.values

#####
# Note, this is the likely line to toggle between commented<->uncommented if seeing issues
#####
# Shift records not containing the year to the right by one to line things up
gp_detail.iloc[rowsToShift] = gp_detail.iloc[rowsToShift].apply(lambda v: pd.Series([None] + v.to_list()[:-1]), axis=1)

gp_detail.iloc[rowsToShift, 0] = pd.to_numeric('', errors='coerce')
""";

gp_detail = gp_detail_raw.copy()
gp_detail.columns = cols2

# Fill the years for the rest of the rows
gp_detail['Year'] = gp_detail['Year'].ffill(limit=8)

#print(gp_detail.head(20))

df_ga_detail_melted = gp_detail.melt(id_vars=["Key", "Year"])

df_ga_detail_melted = df_ga_detail_melted[df_ga_detail_melted["variable"] != "Total"]

df_ga_detail_melted['dt'] = pd.to_datetime(df_ga_detail_melted['Year'].apply(str).apply(lambda v: v.split(".")[0]) +\
                                           '-' + df_ga_detail_melted['variable'], format="%Y-%b", errors='coerce')

df_ga_detail_melted['value'] = df_ga_detail_melted['value']\
                                    .apply(lambda v: str(v).replace(",", ""))\
                                    .apply(lambda v: pd.to_numeric(v, errors='coerce'))

#print(df_ga_detail_melted.fillna('nan').dropna().tail())

df_ga_ts = df_ga_detail_melted[['dt', 'Key', 'value']].dropna().pivot(index='dt', columns='Key').reset_index()
df_ga_ts.columns = ['dt', 'Export Empty', 'Export Full', 'Export Total',\
                    'Import Empty', 'Import Full', 'Import Total', 'Total All', 'Total Empty', 'Total Full']

df_ga_ts = df_ga_ts.dropna()

In [74]:
doYoyChart(df_ga_ts, location='Georgia', metric='Import Empty')

alt.Chart(...)

In [76]:
doYoyChart(df_ga_ts, location='Georgia', metric='Import Full')

alt.Chart(...)

In [77]:
doYoyChart(df_ga_ts, location='Georgia', metric='Export Total')

alt.Chart(...)

In [78]:
doYoyChart(df_ga_ts, location='Georgia', metric='Export Empty')

alt.Chart(...)

In [79]:
doYoyChart(df_ga_ts, location='Georgia', metric='Export Full')

alt.Chart(...)

## Northwest Seaport Alliance (Ports of Seattle and Tacoma)

In [80]:
rawPg = !curl -s 'https://www.nwseaportalliance.com/stats-stories/cargo-stats?page=0' | grep '<span>1 of'

pageCount = re.findall("of ([0-9]+)", "".join(rawPg))[0]

#print(pageCount)

regex = '<a href="([^"]+)"(?:[^>]*)?>Cargo statistics (?:–|-)(?:nbsp;)?\s?([A-z]+)\s([0-9]+)</a>'

links = []
# Last page has a different format
for p in range(int(pageCount)-1):
    base = 'https://www.nwseaportalliance.com/stats-stories/cargo-stats?page={}'.format(p)
    rawUrl = !curl -s $base | egrep '(href|pdf)' | egrep '[C,c]argo stat'
    
    urlsAndDates = re.findall(regex, "".join(rawUrl))
    
    links = links + [u[0] if "https" in u[0] else "https://www.nwseaportalliance.com" + u[0] for u in urlsAndDates]
    #print(urlsAndDates)

In [81]:
@memory.cache
def fetchAndParseNwSeaportReport(url, retries=5):
    if retries < 0:
        return None
    try:
        req = Request(url, data=None, headers={ 'User-Agent': uaString })

        data = urlopen(req)
  
        tmp = read_pdf(BytesIO(data.read()), pages='all')[0]

        #print(tmp.iloc[:, 0].map(lambda v: re.sub("[\d,]", "", str(v)).strip()).values)
        if tmp.iloc[0, 1] == "Int'l Import full TEUs":
            #print('one')
            tmp.columns = ['Key'] + tmp.iloc[0, 1:].apply(lambda v: str(v).replace("\r", " ").strip()).to_list()

            tmp = tmp.iloc[1:].set_index("Key").T
        
        elif "Break Bulk" in tmp.iloc[:, 0].map(lambda v: re.sub("[\d,]", "", str(v)).strip()).values:

            if pd.isnull(tmp.iloc[0, 0]) and pd.isnull(tmp.iloc[0, 1]):
                #print('two')
                tmp = tmp.iloc[1:, :]
                
                tmp.columns = ['Key'] + tmp.iloc[0, 1:].apply(lambda v: str(v).strip()).to_list()

                tmp = tmp.set_index("Key").iloc[1:].T
                
            elif pd.isnull(tmp.iloc[0, 0]) and not pd.isnull(tmp.iloc[0, 1]):
                #print('twotwo')
                tmp.columns = ['Key'] + tmp.iloc[0, 1:].apply(lambda v: str(v).strip()).to_list()

                tmp = tmp.set_index("Key").iloc[1:].T
            
            else:
                #print('three')
                tmp.columns = ['Key'] + tmp.iloc[0, 1:].apply(lambda v: str(v).strip()).to_list()

                tmp = tmp.set_index("Key").iloc[1:].T
            
        else:
            
            tmp.columns = ['Key'] + tmp.iloc[1, 1:].apply(lambda v: str(v).strip()).to_list()

            #print(tmp.iloc[:4, :])
            
            # Guess the row with the year
            if tmp.iloc[1, -3:-1].isna().all():
                #print('fourone')
                tmp = tmp.set_index("Key")
                tmp.iloc[1, :] = None
                tmp.iloc[2, :] = None
                tmp = tmp.dropna(how='all').T
                tmp.index = tmp.iloc[:, 0].values
                tmp = tmp.drop([tmp.columns[0]], axis=1)
                #tmp = pd.concat([tmp.iloc[0, 1:], tmp.iloc[3:, 1:]]).T
            else:
                #print('fourtwo')
                tmp = tmp.set_index("Key").iloc[2:, 1:].T
        
        tmp.columns = tmp.columns.map(lambda v: re.sub("[\d,]", "", str(v)).strip())
        
        tmp = tmp.rename({ "Break Bulk": "Breakbulk" }, axis=1)
        
        if 'Grain' not in tmp.columns:
            tmp['Grain'] = [''] * tmp.shape[0]
            
        if 'Gypsum' not in tmp.columns:
            tmp['Gypsum'] = [''] * tmp.shape[0]
            
        if 'Vessel Calls' not in tmp.columns:
            tmp['Vessel Calls'] = [''] * tmp.shape[0]
            
        tmp['src'] = [url] * tmp.shape[0]
        return tmp
    except HTTPError:
        print("Failed: " + url)
        return fetchAndParseNwSeaportReport(url, retries-1)
    except KeyError:
        print("Failed: " + url)
        return fetchAndParseNwSeaportReport(url, retries-1)

In [82]:
%%time
%%capture

sea_dfs = []
for u in links:
    sea_dfs.append(fetchAndParseNwSeaportReport(u))

sea_df = pd.concat(sea_dfs)

sea_df['dt'] = sea_df.index.map(lambda v: pd.to_datetime("12-31-" + str(v), format='%m-%d-%Y', errors='coerce'))

sea_df.columns = sea_df.columns.map(lambda s: s.replace("'", ""))
sea_df[sea_df.columns.to_list()[:-3]] = sea_df[sea_df.columns.to_list()[:-3]]\
        .applymap(lambda v: pd.to_numeric(str(v).replace(",", ""), errors='coerce'))

sea_df = sea_df[sea_df['dt'].notna()]
sea_df = sea_df.groupby('dt').agg('mean').reset_index()

#sea_df = sea_df.set_index('dt').resample('1M').mean().interpolate(method='linear').reset_index() #.bfill(limit=11).reset_index()

sea_df = sea_df[sea_df['dt'] < '2014-04-01']

#sea_df.head()

CPU times: user 1.15 s, sys: 36.1 ms, total: 1.19 s
Wall time: 1.22 s


In [83]:
sea_months = pd.concat(sea_dfs)
sea_months = sea_months[~sea_months.index.isna()]
sea_ytd_records = sea_months.index.str.match(".*YTD$")

sea_months.index = sea_months.index.str.replace("\r", " ")

short_year_records = sea_months.index.str.match(".* \d\d YTD$")

sea_months.loc[(sea_ytd_records & short_year_records), 'dt'] = sea_months[(sea_ytd_records & short_year_records)].index\
        .map(lambda v: pd.to_datetime(v, format="%b %y YTD", errors='ignore'))

long_year_records = sea_months.index.str.match(".* \d\d\d\d YTD$")

# Run again with different date format (long year vs short year)
sea_months.loc[(sea_ytd_records & long_year_records), 'dt'] = sea_months[(sea_ytd_records & long_year_records)].index\
        .map(lambda v: pd.to_datetime(v, format="%b %Y YTD", errors='ignore'))

long_yearmonth_records = sea_months.index.str.match(".*(ne|ly) \d\d\d\d YTD$")

# Run again with different date format (long year and long month)
sea_months.loc[(sea_ytd_records & long_yearmonth_records), 'dt'] = sea_months[(sea_ytd_records & long_yearmonth_records)].index\
        .map(lambda v: pd.to_datetime(v, format="%B %Y YTD", errors='ignore'))

In [84]:
sea_ytd_records = sea_months.index.str.match(".*YTD$")

#records_to_diff = (sea_months['dt'].dt.month != 1) & sea_ytd_records

sea_months_d = sea_months.set_index('dt').iloc[:, :-4].iloc[:, 1:].copy()

In [85]:
makeANumber = lambda v: pd.to_numeric(v.replace(",", ""), errors='coerce') if isinstance(v, str) else v
sea_months_dn = sea_months_d.applymap(makeANumber)
    
sea_months_dn = sea_months_dn.reset_index().groupby('dt').agg('mean').reset_index()

sea_months_dn = sea_months_dn.set_index('dt')

records_to_exclude_from_diff = (sea_months_dn.index.month == 1)

sea_months_diff = sea_months_dn.sort_index().diff()

sea_months_diff.loc[records_to_exclude_from_diff, :] = sea_months_dn.loc[records_to_exclude_from_diff, :]

In [86]:
# Delete erroneous diff data
sea_months_diff.loc['2016-11-01'] = None

sea_months_diff = sea_months_diff.dropna(how='all')
sea_months_diff.columns = sea_months_diff.columns.map(lambda s: s.replace("'", ""))

In [87]:
sea_df = sea_months_diff.reset_index() #pd.concat([sea_df, sea_months_diff.reset_index()])

In [88]:
doChart(sea_df, location="Seattle/Tacoma", metric="Intl Export full TEUs")

alt.Chart(...)

In [89]:
doYoyChart(sea_df, metric="Intl Export full TEUs", location="Seattle/Tacoma")

alt.Chart(...)

In [90]:
doChart(sea_df, location="Seattle/Tacoma", metric="Intl Empty TEUs")

alt.Chart(...)

In [91]:
doYoyChart(sea_df, metric="Intl Empty TEUs", location="Seattle/Tacoma")

alt.Chart(...)

## Port of Houston

In [92]:
@memory.cache
def fetchHouston(url, retries=5):
    if retries < 0:
        return pd.DataFrame()
    try:
        req = Request(url, data=None, headers={ 'User-Agent': uaString })

        data = urlopen(req)

        cols = ["Date", "Loaded Imports", "Loaded Exports", "Loaded Total",
                "Empty Imports", "Empty Exports", "Empty Total", "Loaded and Empty Total"]

        dfs_hs = read_pdf(BytesIO(data.read()), pages='all', pandas_options={ 'names': cols }) #.head(20)

        df_hs = pd.concat([dfs_hs[0].iloc[5:]] + dfs_hs[1:])

        df_hs[df_hs.columns.to_list()[1:]] = df_hs[df_hs.columns.to_list()[1:]]\
                .applymap(lambda v: pd.to_numeric(v.replace(",", "") if isinstance(v, str) else v, errors='coerce'))

        df_hs['dt'] = df_hs['Date'].map(lambda v: pd.to_datetime(str(v), format='%b-%y', errors='coerce'))

        return df_hs
    except KeyError:
        sleep(1)
        return fetchHouston(url, retries-1)

res = !curl -s 'https://porthouston.com/about-us/statistics/' | grep 'TEU-stats'
    
currentUrl = re.findall('href="([^"]+)"', "".join(res))[0]

#print(currentUrl)
    
# TODO - automate detection of this url from here: https://porthouston.com/about-us/statistics/
df_hs = fetchHouston(currentUrl)
#df_hs.tail()

In [93]:
doChart(df_hs, metric="Empty Imports", location="Houston")

alt.Chart(...)

In [94]:
doChart(df_hs, metric="Empty Exports", location="Houston")

alt.Chart(...)

In [95]:
doYoyChart(df_hs, metric="Empty Imports", location="Houston")

alt.Chart(...)

In [96]:
doYoyChart(df_hs, metric="Empty Exports", location="Houston")

alt.Chart(...)

In [97]:
doYoyChart(df_hs, metric="Loaded Total", location="Houston")

alt.Chart(...)

## Port of Charleston

In [98]:
def fetchCharleston(url, retries=5):
    if retries < 0:
        return None
    try:
        req = Request(url, data=None, headers={ 'User-Agent': uaString })

        data = urlopen(req, timeout=5)

        dfs_ch = read_pdf(BytesIO(data.read()), pages='all')

        # noop
        dfs_ch[0]
        
        return dfs_ch
    except KeyError:
        print("Response code: {}".format(data.getcode()))
        sleep(1)
        return fetchCharleston(url, retries-1)
    except URLError:
        sleep(1)
        return fetchCharleston(url, retries-1)

dfs_ch = fetchCharleston("http://scspa.com/wp-content/uploads/gl078-pc-teu-history.pdf")

In [99]:
def fixColumns(df_):
    firstDataRow = 2
    
    df = df_.copy()
     
    # Standardize column count
    if df.shape[1] == 10:
        # Standardize rail moves column location
        if df.iloc[:, 1].isna().all():
            df.iloc[:, 1] = df.iloc[:, 2]
        
        df = df.drop(df.columns[2], axis=1)
    
    res = df.iloc[firstDataRow:].copy()
    
    #print(df.iloc[firstDataRow:, 2].values)
    
    # split rail column 
    _ = df.iloc[firstDataRow:, 2].str.split(" ", n = 1, expand = True)
    
    #print(_)

    res["RAIL-MOVES"]= _[0]

    res["RAIL-RATIO"]= _[1]

    #res = pd.concat([df.iloc[firstDataRow:, 0:4], res.iloc[:, 2:]], axis=1)

    # split pier loaded column 
    _ = df.iloc[firstDataRow:, 3].str.split(" ", n = 2, expand = True) 

    res["PIER-LOADED-EXPORT"]= _[0] + _[1] 

    res["PIER-LOADED-IMPORT"]= _[2] 

    #res = pd.concat([res, res.iloc[:, -2:]], axis=1)

    try:
        # split pier empty column 
        _ = df.iloc[firstDataRow:, 4].str.split(" ", n = 2, expand = True) 

        res["PIER-EMPTY-EXPORT"]= _[0] + _[1]

        res["PIER-EMPTY-IMPORT"]= _[2]

    except KeyError:
        res["PIER-EMPTY-EXPORT"]= _[0]

        res["PIER-EMPTY-IMPORT"]= _[1]
        
    #res["PIER-TOTAL"]= _[2] 

    #res = pd.concat([df.iloc[firstDataRow:, 0], res.iloc[:, 2:]], axis=1)

    # split loaded teu column 
    _ = df.iloc[firstDataRow:, 6].str.split(" ", n = 2, expand = True) 

    res["TEU-LOADED-EXPORT"]= _[0] + _[1]

    res["TEU-LOADED-IMPORT"]= _[2]

    #res = pd.concat([df.iloc[firstDataRow:, 0], df.iloc[firstDataRow:, 4:6], res.iloc[:, 8:10], df.iloc[firstDataRow:, -1]], axis=1)

    #print(res)

    try:
        # split empty teu column 
        _ = df.iloc[firstDataRow:, 7].str.split(" ", n = 2, expand = True) 

        res["TEU-EMPTY-EXPORT"]= _[0] + _[1]

        res["TEU-EMPTY-IMPORT"]= _[2]
    except KeyError:
        res["TEU-EMPTY-EXPORT"]= _[0]

        res["TEU-EMPTY-IMPORT"]= _[1]

    #res = pd.concat([df.iloc[firstDataRow:, 0], df.iloc[firstDataRow:, 4:6], res.iloc[:, 8:], df.iloc[firstDataRow:, -1]], axis=1)

    res = pd.concat([res.iloc[:, 0],
                     res.iloc[:, 1],
                     res["RAIL-MOVES"],
                     res["RAIL-RATIO"],
                     res["PIER-LOADED-EXPORT"],
                     res["PIER-LOADED-IMPORT"],
                     res["PIER-EMPTY-EXPORT"],
                     res["PIER-EMPTY-IMPORT"],
                     res.iloc[:, 5],
                     res["TEU-LOADED-EXPORT"],
                     res["TEU-LOADED-IMPORT"],
                     res["TEU-EMPTY-EXPORT"],
                     res["TEU-EMPTY-IMPORT"],
                     res.iloc[:, 8],
                    ], axis=1)
    
    res.columns = ['DATE',
                   'RAIL-DRAYS', 'RAIL-MOVES', 'RAIL-RATIO',
                   'PIER-LOADED-EXPORT', 'PIER-LOADED-IMPORT',
                   'PIER-EMPTY-EXPORT', 'PIER-EMPTY-IMPORT',
                   'PIER-TOTAL',
                   'TEU-LOADED-EXPORT', 'TEU-LOADED-IMPORT',
                   'TEU-EMPTY-EXPORT', 'TEU-EMPTY-IMPORT',
                   'TEU-TOTAL']
    #print(res)
    
    # fix remaining column labels
    #res.columns = ['DATE', 'RAIL-DRAYS', 'RAIL-MOVES'] + res.columns.to_list()[3:-1] + ["TEU-TOTAL"]

    # convert all columns to numeric
    res.iloc[:, 1:3] = res.iloc[:, 1:3]\
            .applymap(lambda v: pd.to_numeric(v.replace(",", "").replace(" ", "") if isinstance(v, str) else v, errors='coerce'))

    
    res.iloc[:, 4:] = res.iloc[:, 4:]\
            .applymap(lambda v: pd.to_numeric(v.replace(",", "").replace(" ", "") if isinstance(v, str) else v, errors='coerce'))

    res['dt'] = res['DATE'].apply(lambda v: pd.to_datetime(v, format="%B, %Y", errors='coerce'))

    return res

df_ch = pd.concat([fixColumns(d) for d in dfs_ch])

#df_ch.head()


In [100]:
doChart(df_ch, metric='TEU-EMPTY-EXPORT', location='Charleston')

alt.Chart(...)

In [101]:
doChart(df_ch, metric='TEU-LOADED-IMPORT', location='Charleston')

alt.Chart(...)

In [102]:
doChart(df_ch, metric='TEU-TOTAL', location='Charleston')

alt.Chart(...)

In [103]:
doYoyChart(df_ch, metric='TEU-LOADED-EXPORT', location='Charleston')

alt.Chart(...)

In [104]:
doYoyChart(df_ch, metric='TEU-LOADED-IMPORT', location='Charleston')

alt.Chart(...)

In [105]:
doYoyChart(df_ch, metric='TEU-TOTAL', location='Charleston')

alt.Chart(...)

## Port of Virginia

In [106]:
# http://www.portofvirginia.com/who-we-are/port-stats/

result = !curl -s 'http://www.portofvirginia.com/who-we-are/port-stats/' | grep 'xlsx'

currentReport = [u for u in re.findall('href="([^"]+)"', "".join(result)) if "xls" in u][0]

#print(currentReport)

oldUrl = "http://www.portofvirginia.com/excel/Port_of_Virginia_Statistics.xlsx"

url = currentReport #"https://web.archive.org/web/20191218025738if_/http://www.portofvirginia.com/excel/Port_of_Virginia_Statistics.xlsx"
df_va = pd.read_excel(url, skiprows=1)
df_va.columns = ['Month', 'Key'] + df_va.columns.to_list()[2:]

df_va = df_va.iloc[:71]
df_va = df_va.dropna(how='all')
df_va['Month'] = df_va['Month'].ffill(limit=4)

#df_va.head(20)

df_va_melted = df_va.melt(id_vars=['Month', 'Key'])
df_va_melted['dt'] = pd.to_datetime(df_va_melted['Month'] + ' ' + df_va_melted['variable'].apply(str), format="%b %Y")

df_va_clean = df_va_melted.pivot(index='dt', columns='Key', values='value')

In [107]:
doChart(df_va_clean, location='Virgina', metric='Import Empties')

alt.Chart(...)

In [108]:
doChart(df_va_clean, location='Virgina', metric='Export Empties')

alt.Chart(...)

In [109]:
doChart(df_va_clean, location='Virgina', metric='Import Loads')

alt.Chart(...)

In [110]:
doChart(df_va_clean, location='Virgina', metric='Export Loads')

alt.Chart(...)

In [111]:
doChart(df_va_clean, location='Virgina', metric='Total TEUs')

alt.Chart(...)

In [112]:
doYoyChart(df_va_clean, location='Virgina', metric='Import Loads')

alt.Chart(...)

In [113]:
doYoyChart(df_va_clean, location='Virgina', metric='Export Loads')

alt.Chart(...)

In [114]:
doYoyChart(df_va_clean, location='Virgina', metric='Total TEUs')

alt.Chart(...)

In [115]:
# https://www.oaklandseaport.com/performance/facts-figures/

df_oakland = pd.read_excel("https://www.oaklandseaport.com/files/PDF/Oakland%20Monthly%20TEUs%20July%201997-2019.xlsx",
                           skiprows=5, index_col=0, sheet_name=None)

labels = [['Full', 'Full', 'Full', 'Empty', 'Empty', 'Empty', 'All'],
          ['Inbound', 'Outbound', 'Total', 'Inbound', 'Outbound', 'Total', 'Total']]

df_oak_combined = pd.concat(
   v.iloc[:-1] for v in df_oakland.values()
)

df_oak_combined.columns = map(lambda v: "-".join([*v]), zip(*labels))

df_oak_combined['dt'] = df_oak_combined.index.map(
    lambda v: pd.to_datetime(str(v), format="%Y-%m-%d 00:00:00", errors='coerce')
)

df_oak_combined = df_oak_combined.iloc[6:]

df_oak_combined = df_oak_combined.set_index('dt').resample('2M').mean()

In [116]:
command = "curl -s 'https://www.oaklandseaport.com/performance/facts-figures/' | grep 'Container Activity' | awk -F 'text-2-columns\">' '{ print $2}' | awk -F '<h2' '{ print $1 }'"

df_oak_latest_res = !$command

df_oak_latest_raw = "".join(df_oak_latest_res)

extractMonthAndValue = lambda s: re.findall('title="([A-z]+)"><span class="number">([0-9,.]+)', s)

def getLabel(n):
    if n == 0:
        return 'Full-Inbound'
    elif n == 1:
        return 'Full-Outbound'
    elif n == 2:
        return 'Empty-Inbound'
    else:
        return 'Empty-Outbound' 

df_oak_latest_stg = [(getLabel(n-1), extractMonthAndValue(v)) for n, v in enumerate(re.split('class="chart-subtitle"', df_oak_latest_raw)) if "number" in v]

df_oak_latest = pd.DataFrame()
for kind, vals in df_oak_latest_stg:
    #print(kind, vals)
    tmp = pd.DataFrame.from_records(vals)
    tmp['dt'] = pd.to_datetime("2020-" + tmp.iloc[:, 0], format="%Y-%b")
    tmp.iloc[:, 1] = pd.to_numeric(tmp.iloc[:, 1].map(lambda v: v.replace(",", "")))
    tmp['type'] = tmp.index.map(lambda v: kind)
    tmp.columns = ['month', 'value', 'dt', 'type']
    df_oak_latest = pd.concat([df_oak_latest, tmp])

In [117]:
df_oak_latest_tab = df_oak_latest.pivot_table(index='dt', columns='type', values='value')

df_oak_latest_tab['Full-Total'] = df_oak_latest_tab['Full-Inbound'] + df_oak_latest_tab['Full-Outbound']
df_oak_latest_tab['Empty-Total'] = df_oak_latest_tab['Empty-Inbound'] + df_oak_latest_tab['Empty-Outbound']
df_oak_latest_tab['All-Total'] = df_oak_latest_tab['Full-Total'] + df_oak_latest_tab['Empty-Total']

#df_oak_latest_tab.head()

In [118]:
df_oak_combined = pd.concat([df_oak_combined, df_oak_latest_tab])

## Port of Oakland

In [119]:
doChart(df_oak_combined, location='Oakland')

alt.Chart(...)

In [120]:
doYoyChart(df_oak_combined, location='Oakland')

alt.Chart(...)

In [121]:
doChart(df_oak_combined, metric='Empty-Total', location='Oakland')

alt.Chart(...)

In [122]:
doYoyChart(df_oak_combined, metric='Empty-Total', location='Oakland')

alt.Chart(...)

In [123]:
doChart(df_oak_combined, metric='All-Total', location='Oakland')

alt.Chart(...)

In [124]:
doYoyChart(df_oak_combined, metric='All-Total', location='Oakland')

alt.Chart(...)